In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import os
import pandas as pd
import seaborn as sns
%matplotlib inline

In [ ]:
os.chdir(r'Z:\ChiPrivate\Chicago Reports and Evaluation\SY18\Eval Management\ACM_analysis')

In [ ]:
heatmaps_df = pd.read_csv('OC_clean.csv')
tut_time_df = pd.read_csv('time_on_task_2017-12-11.csv')
assmt_df = pd.read_csv('assessment_growth.csv')
survey_df = pd.read_csv('ACM_Q2_survey.csv')

In [ ]:
assmt_df['Assessment Type'] = assmt_df['Assessment Type'].str.replace('NWEA - ELA', 'Tutoring: Literacy')
assmt_df['Assessment Type'] = assmt_df['Assessment Type'].str.replace('NWEA - MATH', 'Tutoring: Math')
assmt_df['Key'] = assmt_df['Student__c'] + assmt_df['Assessment Type']

tut_time_df['Key'] = tut_time_df['Student__c'] + tut_time_df['Program__c_Name']

df = assmt_df[['Key', 'Assessment Type', 'Growth_v_Target']].merge(tut_time_df, how='inner', on='Key')

df = df.loc[~df['Growth_v_Target'].isnull()
            &~df['Amount_of_Time__c_YTD'].isnull()]

df['Hit_Target?'] = 0
df['Hit_Target?'].loc[df['Growth_v_Target']>=0] = 1

mean_df = df.groupby('Staff__c').agg(['mean', 'std'])

mean_df.columns = [' '.join(col).strip() for col in mean_df.columns.values]

mean_df.reset_index(inplace=True)
df = mean_df.merge(survey_df, on='Staff__c')

In [ ]:
from sklearn_pandas import DataFrameMapper
import numpy as np
import sklearn.preprocessing, sklearn.decomposition, sklearn.linear_model, sklearn.pipeline, sklearn.metrics
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_selection import SelectKBest, chi2

In [ ]:
data = df.copy()

### TODO:
1. decide what attributes I want to pass in
    * dimensionality reduction? represent the gist of a survey set with less data
    * all of them? or at least the nominal ones? would need to clean all of them, but otherwise not unreasonable
    * what's the role of common sense?
* Decide what scoring method to best select attributes
* Create decision tree
* Test other measures of ACM effectiveness

In [ ]:
mapper = DataFrameMapper([('NPS', sklearn.preprocessing.LabelBinarizer()),
                          ('Growth_v_Target mean', None)], df_out=True)

In [ ]:
mapper

In [ ]:
mapper_fs = DataFrameMapper([(['children','salary'], SelectKBest(chi2, k=1))])
mapper_fs.fit_transform(data[['children','salary']], data['Growth_v_Target mean'])

In [ ]:
from sklearn import tree

In [ ]:
X = [[0, 0], [1, 1]]
Y = [0, 1]
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X, Y)

In [ ]:
sns.jointplot(x=("Hit_Target?", "mean"), y=("Amount_of_Time__c_YTD", "mean"), data=mean_df, kind="kde")

In [ ]:
sns.jointplot(x=("Growth_v_Target", "mean"), y=("Amount_of_Time__c_YTD", "mean"), data=mean_df, kind="kde", xlim=(-20, 20), ylim=(0, 1200))

In [ ]:
mean_df[("Growth_v_Target", "mean")].hist()

In [ ]:
mean_df[("Hit_Target?", "mean")].hist()

## Analysis Questions
* do SYACMs have greater impact? do they have greater impact when working with the same students?